In [3]:
import os

In [4]:
%pwd

'/home/debasish/home/Indian-Coin-Detection/research'

In [5]:
os.chdir('..//')

In [6]:
%pwd

'/home/debasish/home/Indian-Coin-Detection'

Following the project workflows

In [5]:
# Entity is the return type of the function

from dataclasses import dataclass
from pathlib import Path

# Creating the entity class
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from Indian_Coin_Detection.constants import *
from Indian_Coin_Detection.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [1]:
import os
import urllib.request as request
import zipfile
from Indian_Coin_Detection import logger
from Indian_Coin_Detection.utils.common import  get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
# Whole Pipeline
try:
    config = ConfigurationManager() #First initializing the configuration manager
    data_ingestion_config = config.get_data_ingestion_config()
    #Calling the components which takes dataingestjion config as input
    data_ingestion = DataIngestion(config=data_ingestion_config) 
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-09-05 13:18:58,547: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-09-05 13:18:58,551: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-05 13:18:58,554: INFO: common: created directory at: artifacts]
[2023-09-05 13:18:58,556: INFO: common: created directory at: artifacts/data_ingestion]
[2023-09-05 13:18:59,804: INFO: 1133211290: temp download! with following info: 
Server: GitHub.com
Date: Tue, 05 Sep 2023 07:48:59 GMT
Content-Type: text/html; charset=utf-8
Vary: X-PJAX, X-PJAX-Container, Turbo-Visit, Turbo-Frame, Accept-Encoding, Accept, X-Requested-With
ETag: W/"34d17fd4bab8b8fdf1f5b5a05fb39bb4"
Cache-Control: max-age=0, private, must-revalidate
Strict-Transport-Security: max-age=31536000; includeSubdomains; preload
X-Frame-Options: deny
X-Content-Type-Options: nosniff
X-XSS-Protection: 0
Referrer-Policy: no-referrer-when-downgrade
Content-Security-Policy: default-src 'none'; base-uri 'self'; child-src github.com/assets-cdn/worker/ gis

In [ ]:
import requests
import zipfile
import io

# Replace with the URL of the GitHub zip file you want to download
github_zip_url = 'https://github.com/pradhandebasish2046/Indian-Coin-Detection/blob/main/coin-data-small.zip'

# Define the local file name to save the downloaded zip file
local_zip_filename = 'repo-master.zip'

# Send a GET request to download the zip file from GitHub
response = requests.get(github_zip_url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Open the local file for writing in binary mode
    with open(local_zip_filename, 'wb') as zip_file:
        # Write the content of the response to the local file
        zip_file.write(response.content)

    # Extract the downloaded zip file
    with zipfile.ZipFile(local_zip_filename, 'r') as zip_ref:
        # Replace 'folder_name' with the name of the folder you want to extract
        zip_ref.extractall()

    print(f'Successfully downloaded and extracted {local_zip_filename}')
else:
    print(f'Failed to download the zip file (Status code: {response.status_code})')
